In [ ]:
YDSHPERE = False

In [ ]:
if YDSHPERE:
    %pip install efficientnet_pytorch
else:
    !pwd

In [ ]:
#!g1.1
import os
import gc
import sys
import cv2
import glob
import time
import json
import random
import pandas as pd
import numpy as np
import torch
import torchvision.models as models
import torch.nn as nn
from tqdm.notebook import tqdm
from efficientnet_pytorch import model as enet

In [ ]:
KAGGLE = False
if not KAGGLE: os.environ['CUDA_VISIBLE_DEVICES'] = '1' 
else: pass
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('GPU is available')
else:
    DEVICE = torch.device('cpu')
    print('CPU is used')

In [ ]:
DEBUG = True
VER = 'v2'
if KAGGLE:
    DATA_PATH = '../input/seti-breakthrough-listen'
    MDLS_PATH = f'../input/seti-models-{VER}'
else:
    DATA_PATH = './data'
    MDLS_PATH = f'./models_{VER}'
TH = None
TTAS = [0]
FOLDS = [0, 1, 2, 3]
IMGS_PATH = f'{DATA_PATH}/test'

start_time = time.time()

In [ ]:
if DEBUG:
    df_subm = pd.read_csv(f'{DATA_PATH}/sample_submission.csv').sample(100).reset_index(drop=True)
else:
    df_subm = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')
print (df_subm.shape)
df_subm['img_path'] = df_subm['id'].apply(lambda x: f'{IMGS_PATH}/{x[0]}/{x}.npy')
df_subm.head()

In [ ]:
with open(f'{MDLS_PATH}/params.json') as file:
    params = json.load(file)
WORKERS = 2 if KAGGLE else 8 #params['workers']
print('loaded params:', params, '\n')

if not TH:
    with open(f'{MDLS_PATH}/th.json') as file:
        th = json.load(file)
    print('threshold:', th)
else: 
    print('threshold:', TH)

In [ ]:
def flip(img, axis=0):
    if axis == 1:
        return img[:, ::-1, :]
    elif axis == 2:
        return img[:, :, ::-1]
    elif axis == 3:
        return img[:, ::-1, ::-1]
    else:
        return img

class ClassificationDataset:
    
    def __init__(self, img_paths, targets, tta): 
        self.img_paths = img_paths
        self.targets = targets
        self.tta = tta

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, item):      
        img = np.load(self.img_paths[item]).astype(np.float32)
        img = flip(img, axis=self.tta)
        targets = self.targets[item]        
        return {
            'image': torch.tensor(img.copy(), dtype=torch.float),
            'targets': torch.tensor(targets, dtype=torch.float),
        }

class EffNet(nn.Module):
    
    def __init__(self, params, out_dim):
        super(EffNet, self).__init__()
        self.enet = enet.EfficientNet.from_name(params['backbone'])
        nc = self.enet._fc.in_features
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(
            in_channels=6, 
            out_channels=3, 
            kernel_size=3, 
            stride=1, 
            padding=3, 
            bias=False
        )
        self.myfc = nn.Sequential(
            nn.Dropout(params['dropout']),
            nn.Linear(nc, int(nc / 4)),
            nn.Dropout(params['dropout']),
            nn.Linear(int(nc / 4), out_dim)
        )
    
    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [ ]:
models = []
for n_fold in FOLDS:
    model = EffNet(params=params, out_dim=1) 
    path = '{}/model_best_{}.pth'.format(MDLS_PATH, n_fold)
    state_dict = torch.load(path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda(DEVICE)
    models.append(model)
    print('loaded:', path)
del state_dict, model
gc.collect();

In [ ]:
datasets, loaders = [], []
for tta in TTAS:
    subm_imgs = df_subm.loc[:, 'img_path'].values
    subm_targets = df_subm.loc[:, 'target'].values
    dataset = ClassificationDataset(
        img_paths=subm_imgs, 
        targets=subm_targets,
        tta=tta
    )
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=params['batch_size'],
        shuffle=False, 
        num_workers=WORKERS
    )
    datasets.append(dataset)
    loaders.append(loader)

In [ ]:
logits = []
with torch.no_grad():
    for i, model in enumerate(models):
        for j, loader in enumerate(loaders):
            logits_tta = []
            for data in tqdm(loader, desc=f'model {i}, loader {j}'):
                img_data = data['image']
                img_data = img_data.to(DEVICE)
                preds = np.squeeze(model(img_data).sigmoid().cpu().numpy())
                logits_tta.extend(preds.tolist())
            logits.append(logits_tta)
logits = np.mean(logits, axis=0)
logits = np.squeeze(np.vstack(logits))

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
df_subm.loc[:, 'target'] = logits
df_subm.drop(['img_path'], axis=1, inplace=True)
print(df_subm.shape)
df_subm.head()

In [ ]:
df_subm.to_csv('submission.csv', index=False)